In [1]:
import os
from pathlib import Path
from typing import List, Union
import ast
from datetime import datetime

def find_python_files(package_path: Union[str, Path]) -> List[str]:
    if isinstance(package_path, str):
        package_path = Path(package_path)
    
    python_files = []
    if not package_path.exists():
        raise FileNotFoundError(f"경로를 찾을 수 없습니다: {package_path}")
    
    for root, dirs, files in os.walk(package_path):
        if '__pycache__' in dirs:
            dirs.remove('__pycache__')
        for file in files:
            if file.endswith('.py') and file not in ["merged_output.py"] and not file.startswith("test_"):
                full_path = os.path.join(root, file)
                python_files.append(os.path.abspath(full_path))
    
    return sorted(python_files)

def merge_python_files(files: List[str], output_path: str) -> None:
    merged_content = [f"# Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"]
    imports = set()
    
    for file_path in files:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        file_name = os.path.basename(file_path)
        merged_content.append(f"\n# ===== {file_name} =====\n")
        
        try:
            tree = ast.parse(content)
            for node in ast.walk(tree):
                if isinstance(node, (ast.Import, ast.ImportFrom)):
                    imports.add(ast.unparse(node))
            
            merged_content.append(content)
        except Exception as e:
            print(f"Warning: {file_path} 처리 중 오류 발생: {str(e)}")
            merged_content.append(content)
    
    imports_content = "\n".join(sorted(imports)) + "\n\n" if imports else ""
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(imports_content + "".join(merged_content))
        print("merge completed.")


package_path = '..'
output_file = 'merged_output.py'

files = find_python_files(package_path)
merge_python_files(files, output_file)

merge completed.
